In [0]:
import pandas as pd
import numpy as np
import io

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
link = 'https://drive.google.com/open?id=1YVFiEhU6BixPoHL5oMOEIAbA1pWM99CV'
fluff, id = link.split('=')
print (id)

1YVFiEhU6BixPoHL5oMOEIAbA1pWM99CV


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Kindle_Book_Dataset.csv')  
Books = pd.read_csv('Kindle_Book_Dataset.csv')

In [11]:
Books.head()

,url,title,author,price,save,pages,description,size,publisher,language,text_to_speech,x_ray,lending,customer_reviews,stars
0,http://www.amazon.com/How-Bartender-Bartending...,How to be a Bartender: Get a Bartending Job Wi...,Dave Allred,9.99,NaN,171.0,"In his revolutionary new book, Dave Allred (Th...",249.0,NaN,English,Enabled,Not Enabled,Enabled,23.0,4.9
1,http://www.amazon.com/Raise-Bar-Action-Based-C...,Raise the Bar: An Action-Based Method for Maxi...,Jon Taffer,2.00,24.00,258.0,If there’s anyone who can prevent a bar or res...,1427.0,Amazon Publishing,English,Enabled,Enabled,Enabled,80.0,4.7
2,http://www.amazon.com/The-Professional-Bartend...,The Professional Bartender's Handbook: A Recip...,Valerie Mellema,9.87,12.08,288.0,The Professional Bartenders Handbook is writte...,5643.0,Atlantic Publishing Group Inc.; illustrated ed...,English,Enabled,Not Enabled,Not Enabled,32.0,4.2
3,http://www.amazon.com/Double-Your-Tips-Jerry-M...,How to Double Your Tips [Kindle Edition],Jerry Minchey,2.99,NaN,36.0,"How to Double Your Tips (For every waitress, w...",98.0,Stony River Media,English,Enabled,Not Enabled,Enabled,6.0,4.5
4,http://www.amazon.com/The-Bartenders-Bible-Mix...,The Bartender's Bible: 1001 Mixed Drinks [Kind...,Gary Regan,5.99,2.00,400.0,Mix Drinks Like A Pro Now you can with this in...,647.0,HarperCollins e-books,English,Enabled,Not Enabled,Not Enabled,59.0,3.6


In [12]:
print(Books.shape)

(49197, 15)


In [13]:
Books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49197 entries, 0 to 49196
Data columns (total 15 columns):
url                 49197 non-null object
title               49196 non-null object
author              49187 non-null object
price               48642 non-null float64
save                36290 non-null float64
pages               48987 non-null float64
description         49197 non-null object
size                48155 non-null float64
publisher           41652 non-null object
language            49100 non-null object
text_to_speech      47675 non-null object
x_ray               47686 non-null object
lending             47767 non-null object
customer_reviews    47751 non-null float64
stars               47752 non-null float64
dtypes: float64(6), object(9)
memory usage: 5.6+ MB


In [0]:
books= Books.drop(Books.index[20000:49197], axis=0)

In [23]:
books.tail()

,url,title,author,price,save,pages,description,size,publisher,language,text_to_speech,x_ray,lending,customer_reviews,stars
19995,http://www.amazon.com/Kiss-Before-Dying-Pegasu...,A Kiss Before Dying (Pegasus Crime) [Kindle Ed...,Ira Levin,9.99,4.96,288.0,"A modern classic, this novel set a new standar...",470.0,Pegasus Books,English,Enabled,Enabled,Enabled,58.0,4.5
19996,http://www.amazon.com/Model-World-And-Other-St...,A Model World: And Other Stories [Kindle Edition],Michael Chabon,7.69,6.30,210.0,Chabon’s dazzling first story collection revea...,357.0,Open Road Media,English,Enabled,Not Enabled,Enabled,17.0,4.0
19997,http://www.amazon.com/Werewolves-Their-Youth-M...,Werewolves in Their Youth: Stories [Kindle Edi...,Michael Chabon,7.69,8.31,228.0,"Chabon’s second story collection, about couple...",429.0,Open Road Media; 1 edition,English,Enabled,Not Enabled,Enabled,33.0,3.9
19998,http://www.amazon.com/Mysteries-Pittsburgh-P-S...,The Mysteries of Pittsburgh (P.S.) [Kindle Edi...,Michael Chabon,9.99,4.00,320.0,Chabon’s sensational debut novel: the coming-o...,707.0,Open Road Media,English,Enabled,Not Enabled,Enabled,146.0,3.9
19999,http://www.amazon.com/Perfectly-Reasonable-Dev...,Perfectly Reasonable Deviations From the Beate...,Richard P. Feynman,9.99,8.01,514.0,"A Nobel Prize-winning physicist, a loving husb...",3848.0,Basic Books,English,Enabled,Not Enabled,Not Enabled,41.0,5.0


In [15]:
books_clean = books.drop(columns=['url','save','size','text_to_speech','x_ray','lending','customer_reviews'])
books_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 8 columns):
title          19999 non-null object
author         19998 non-null object
price          19880 non-null float64
pages          19912 non-null float64
description    20000 non-null object
publisher      16838 non-null object
language       19984 non-null object
stars          19688 non-null float64
dtypes: float64(3), object(5)
memory usage: 1.4+ MB


In [16]:
books_clean.head(1)['description']

0    In his revolutionary new book, Dave Allred (Th...
Name: description, dtype: object

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3, max_features=None,
                      strip_accents='unicode', analyzer='word',
                      token_pattern=r'\w{1,}',ngram_range=(1,3),
                      stop_words = 'english')
books_clean['description']= books_clean['description'].fillna('')


In [0]:
tfv_matrix = tfv.fit_transform(books_clean['description'])

In [0]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [0]:
indices = pd.Series(books_clean.index, index=books_clean['title']).drop_duplicates()

In [0]:
def give_rec(title, sig=sig):
  idx = indices[title]
  sig_scores = list(enumerate(sig[idx]))
  sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
  sig_scores = sig_scores[1:11]
  book_indices = [i[0] for i in sig_scores]
  return books_clean['title'].iloc[book_indices]

In [26]:
give_rec('A Model World: And Other Stories [Kindle Edition]')

7302     Maps and Legends: Reading and Writing Along th...
8131                          Wonder Boys [Kindle Edition]
19997    Werewolves in Their Youth: Stories [Kindle Edi...
19998    The Mysteries of Pittsburgh (P.S.) [Kindle Edi...
10960    You Can't Keep a Good Woman Down: Stories [Kin...
7883     The Amazing Adventures of Kavalier & Clay (wit...
17553    Taking Flight!: Master the DISC Styles to Tran...
3242          The One Good Thing: A Novel [Kindle Edition]
2684     The Book of Baseball Literacy: 3rd Edition: Ne...
3620        This Song Will Save Your Life [Kindle Edition]
Name: title, dtype: object

In [0]:
A

IndentationError: unexpected indent (<ipython-input-6-d5200e432aa0>, line 2)